In [1]:
import numpy as np
import pandas as pd

In [18]:
df = (
    pd
    .read_csv("05_flats.csv")
    .drop_duplicates()
    .apply(lambda col: col.str.strip().str.lower() if col.dtype == 'object' else col)
)

In [19]:
df.head(3)

,property_name,link,society,price,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating,property_id
0,2 bhk flat in krishna colony,https://www.99acres.com/2-bhk-bedroom-apartmen...,maa bhagwati residency,45 lac,"₹ 5,000/sq.ft.",carpet area: 900 (83.61 sq.m.),2 bedrooms,2 bathrooms,1 balcony,NaN,"krishna colony, gurgaon, haryana",4th of 4 floors,west,1 to 5 year old,"['chintapurni mandir', 'state bank atm', 'pear...",so with lift.maa bhagwati residency is one of ...,"['3 fan', '4 light', '1 wardrobe', 'no ac', 'n...","['feng shui / vaastu compliant', 'security / f...","['environment4 out of 5', 'safety4 out of 5', ...",c68850746
1,2 bhk flat in ashok vihar,https://www.99acres.com/2-bhk-bedroom-apartmen...,apna enclave,50 lac,"₹ 7,692/sq.ft.",carpet area: 650 (60.39 sq.m.),2 bedrooms,2 bathrooms,1 balcony,NaN,"46b, ashok vihar, gurgaon, haryana",1st of 3 floors,west,10+ year old,"['chintapurni mandir', 'sheetla mata mandir', ...","property situated on main road, railway statio...","['3 wardrobe', '4 fan', '1 exhaust fan', '1 ge...","['security / fire alarm', 'maintenance staff',...","['environment4 out of 5', 'safety4 out of 5', ...",h68850564
2,2 bhk flat in sohna,https://www.99acres.com/2-bhk-bedroom-apartmen...,tulsiani easy in homes,40 lac,"₹ 6,722/sq.ft.",carpet area: 595 (55.28 sq.m.),2 bedrooms,2 bathrooms,3 balconies,NaN,"sohna, gurgaon, haryana",12nd of 14 floors,NaN,0 to 1 year old,"['huda city metro', 'golf course extn road', '...","this property is 15 km away from badshapur, gu...",NaN,"['power back-up', 'feng shui / vaastu complian...","['environment4 out of 5', 'safety4 out of 5', ...",j68850120


In [62]:
df = pd.read_csv("05_flats.csv")
df.drop_duplicates(inplace=True)
df.drop([286], inplace=True)
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].str.strip()
        df[col] = df[col].str.lower()
df['bhk'] = df['property_name'].str.split("bhk").apply(lambda x : x[0])
df['locality'] = df['property_name'].str.split(" in ").apply(lambda x : x[1]).str.strip()
df = df.drop(columns=['property_name'])

In [71]:
df['price'].str.split().apply(lambda x: len(x))

TypeError: object of type 'float' has no len()

In [63]:
df['price_in_crore'] = df.apply(lambda x : x['price']/100 if x['price'].contains('lac') else x['price'])

KeyError: 'price'

In [61]:
df.head(3)

,link,society,price,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,...,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating,property_id,bhk,locality
0,https://www.99acres.com/2-bhk-bedroom-apartmen...,maa bhagwati residency,45 lac,"₹ 5,000/sq.ft.",carpet area: 900 (83.61 sq.m.),2 bedrooms,2 bathrooms,1 balcony,NaN,"krishna colony, gurgaon, haryana",...,west,1 to 5 year old,"['chintapurni mandir', 'state bank atm', 'pear...",so with lift.maa bhagwati residency is one of ...,"['3 fan', '4 light', '1 wardrobe', 'no ac', 'n...","['feng shui / vaastu compliant', 'security / f...","['environment4 out of 5', 'safety4 out of 5', ...",c68850746,2,krishna colony
1,https://www.99acres.com/2-bhk-bedroom-apartmen...,apna enclave,50 lac,"₹ 7,692/sq.ft.",carpet area: 650 (60.39 sq.m.),2 bedrooms,2 bathrooms,1 balcony,NaN,"46b, ashok vihar, gurgaon, haryana",...,west,10+ year old,"['chintapurni mandir', 'sheetla mata mandir', ...","property situated on main road, railway statio...","['3 wardrobe', '4 fan', '1 exhaust fan', '1 ge...","['security / fire alarm', 'maintenance staff',...","['environment4 out of 5', 'safety4 out of 5', ...",h68850564,2,ashok vihar
2,https://www.99acres.com/2-bhk-bedroom-apartmen...,tulsiani easy in homes,40 lac,"₹ 6,722/sq.ft.",carpet area: 595 (55.28 sq.m.),2 bedrooms,2 bathrooms,3 balconies,NaN,"sohna, gurgaon, haryana",...,NaN,0 to 1 year old,"['huda city metro', 'golf course extn road', '...","this property is 15 km away from badshapur, gu...",NaN,"['power back-up', 'feng shui / vaastu complian...","['environment4 out of 5', 'safety4 out of 5', ...",j68850120,2,sohna
